# Exercicis Classificació d'imatges

Hem vist com entrenar una xarxa neuronal per classificar imatges.

Ara ho farem amb un conjunt de dades diferent, per veure si el que hem après és aplicable a altres conjunts de dades.

El conjunt de dades que farem servir és el [sportsMNIST](https://www.kaggle.com/clarencezhao/sports8) de Kaggle. Aquest conjunt de dades conté imatges de diferents esports i el nostre objectiu serà classificar-les i comprovar com es comporta la xarxa amb imatges que no ha vist durant l'entrenament.

Has d'intentar aconsseguir una _validation accuracy_ superior al 80%.

Si no puguessis aconseguir-ho, pots provar de fer servir la tècnica de _data augmentation[1]_ per veure si això millora els resultats.

[1] https://www.tensorflow.org/tutorials/images/data_augmentation

## Carreguem les llibreries

En pprimer lloc instal·lem i carreguem les llibreries que farem servir.

In [1]:
%pip install keras requests

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

import zipfile

import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.utils import image_dataset_from_directory
from keras.layers.experimental.preprocessing import Rescaling

import matplotlib.pyplot as plt
import numpy as np

import requests

## Preparem les dades

Primer de tot, carregarem les dades i les visualitzarem per veure com són.

### Carreguem les dades

El conjunt de dades està comprimit en un fitxer zip. El descomprimirem i crearem un `Dataset` utilitzant `image_dataset_from_directory` de `keras.utils`.

In [ ]:
zip_url = "https://lawer.github.io/mia/apunts/8.-Reconeixement%20imatges/sportimages.zip"

filename = "sportimages.zip"

r = requests.get(zip_url, allow_redirects=True)
open(filename, 'wb').write(r.content)

with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
dataset = image_dataset_from_directory(
    'sportimages',
    image_size=(28, 21),
    validation_split=0.2,
    seed=123,
    subset='both',
    label_mode="int"
)

class_names = dataset[0].class_names
print(class_names)

num_classes = len(class_names)


`image_dataset_from_directory` espera que les imatges estiguin organitzades en carpetes, on cada carpeta és una categoria diferent. Això és el que farem. Ens retornarà un `_PrefetchDataset` que podrem utilitzar per entrenar la xarxa.

El _PrefetchDataset_ és un tipus de dades que permeten carregar les imatges de manera eficient, sense haver de carregar-les totes a la memòria. Solament hem de saber que per poder accedir a les imatges hem d'utilitzar el métode `take`.

Per veure el funcionament  de `take` mostrarem les 9 primeres imatges del conjunt de dades.

In [ ]:
train_ds, val_ds = dataset

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

### Entrenem la xarxa

Ara has d'entrenar la xarxa. Fes les proves en una xarxa _Feed Forward_ primer i després en una xarxa convolucional. En la convolucional ver agregant capes de convolució i _pooling_ poc a poc per veure com afecta a la precisió.

Recorda també normalitzar les dades i fer servir la tècnica de _data augmentation_ si ho consideres necessari.

In [ ]:
model = Sequential([
    Rescaling(1./255, input_shape=(28, 21, 3)),
    Conv2D(16, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes)
])

model.compile(optimizer='adam',
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)